# The pyrtlsdr library

This is the most basic way to control the rtl dongle. First let's sample some data.

In [ ]:
from rtlsdr import RtlSdr

sdr = RtlSdr()

# configure device
sdr.sample_rate = 2.048e6  # Hz
sdr.center_freq = 95e6     # Hz
sdr.freq_correction = 60   # PPM
sdr.gain = 'auto'

result = sdr.read_samples(512)
print(result)
#print(type(result))
#result.shape

Now we can plot the data using the matplotlib library.

In [ ]:
from pylab import *
from rtlsdr import *

sdr = RtlSdr()

# configure device
sdr.sample_rate = 2.4e6
sdr.center_freq = 95e6
sdr.gain = 4

samples = sdr.read_samples(256*1024)
sdr.close()

# use matplotlib to estimate and plot the PSD
psd(samples, NFFT=1024, Fs=sdr.sample_rate/1e6, Fc=sdr.center_freq/1e6, color='red')
xlabel('Frequency (MHz)')
ylabel('Relative power (dB)')

show()

# The rtlobs library:
Let's look at some of the most commonly used things this library can do to collect and post-process IQ data from an RTL-SDR.

In [ ]:
from rtlobs import collect as col
from rtlobs import post_process as post

### Total power
We can find the total power incident on the SDR by integrating the complex samples at a certain frequency for an amount of time:

In [ ]:
# 1024 complex samples per call to the SDR
# 49.6 dB of RtlSdr gain
# 2.4 MHz sample rate/bandwidth
# 1420.6 MHz center frequency
# 10 sec integration time
p_tot = col.run_total_power_int(1024, 49.6, 2.4e6, 1420.4e6, 10)
print('Uncalibrated total power measurement: {}'.format(p_tot))

### Total power calibration 
If p_tot was measured with a telescope pointed at a radio-hot source (e.g. the ground, ~300 Kelvin), and we measure again, this time pointed at a radio-cold source (e.g. empty sky, estimates vary, say ~10 K):

In [ ]:
p_hot = p_tot
p_cold = col.run_total_power_int(1024, 49.6, 2.4e6, 1420.4e6, 10)
print('Uncalibrated total power measurement: {}'.format(p_cold))

We can use these power measurements and assumptions of brightness temperature to perform a "Y-factor" calibration of the system temperature, T_sys, as described in [Single-Dish Calibration Techniques at Radio Wavelengths, K. O'Neil, 2002](http://adsabs.harvard.edu/full/2002ASPC..278..293O):

In [ ]:
from rtlobs import calibrate as cal
t_sys = cal.y_factor_cal(p_hot, p_cold, 300, 10)

If your assumptions are wrong (for instance, pointing at the ceiling then a window, instead of ground, then sky), T_sys might not come out what you expect.

### Spectral lines
We can estimate the power spectral density recorded by the SDR for a given bandwidth by averaging multiple spectra over time to reduce the noise:

In [ ]:
from rtlobs import collect as col
from rtlobs import post_process as post
# 8192 samples per call to the SDR
# 2048 bin resolution power spectral density estimate
# 49.6 dB of RtlSdr gain
# 2.32 MHz sample rate/bandwidth
# 1.420 GHz center frequency
# 10 sec integration time
f, p = col.run_spectrum_int(8192, 2048, 49.6, 2.32e6, 1.420e9, 5)
# Show the figure containing the plotted spectrum
# display(f,p)
fig, ax = post.plot_spectrum(f, p, savefig='../images/spectrum_int.png')

And we can even run in frequency-switching mode, periodically retuning the SDR between two frequencies to reduce baseline bandpass shape and increase signal-to-noise on a spectral line!

Note that the folding process performed on frequency-switched data happens in terms of the absolute values (power) rather than relative values (dB), so we make those changes to the plot labels.

In [ ]:
f, p = col.run_fswitch_int(8192, 2048, 49.6, 2.048e6, 88.95e6, 89.25e6, 5.0, fswitch=5)
fig, ax = post.plot_spectrum(f, p)
# Edit figure labels to be technically correct
ax.set_ylabel('Uncalibrated Power Estimate (V$^2$)')
ax.set_title('Power Spectrum Estimate')
#fig.savefig('../images/fswitch_int.png')